<a href="https://colab.research.google.com/github/diyanigam/Text-to-Website-CampEdUI/blob/main/CampEdUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import re
from collections import defaultdict

def parse_cheatsheet(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    components_by_package = defaultdict(list)
    tag_to_package = {}
    tag_descriptions = {}
    current_package = None
    current_desc = ""

    for line in lines:
        line = line.strip()

        if line.startswith("- @camped-ui/"):
            current_package = line.split()[0][2:]
        elif line.startswith("Description:"):
            current_desc = line.replace("Description:", "").strip()
        elif line.startswith("• <"):
            tag = re.findall(r"<(.*?)>", line)[0]
            components_by_package[current_package].append(tag)
            tag_to_package[tag] = current_package
            tag_descriptions[tag] = current_desc

    return components_by_package, tag_to_package, tag_descriptions

In [22]:
def infer_components_from_prompt(prompt, tag_to_package):
    used_tags = []
    for tag in tag_to_package:
        if tag.lower() in prompt.lower():
            used_tags.append(tag)
    return list(set(used_tags))

def generate_imports(used_tags, tag_to_package):
    pkg_to_tags = defaultdict(list)
    for tag in used_tags:
        pkg = tag_to_package.get(tag)
        if pkg:
            pkg_to_tags[pkg].append(tag)

    import_lines = []
    for pkg, tags in pkg_to_tags.items():
        line = f'import {{ {", ".join(sorted(set(tags))) } }} from "{pkg}";'
        import_lines.append(line)

    return "\n".join(import_lines)

In [24]:
def build_prompt(prompt_text, import_block):
    return f"""
### Instruction:
You are a Next.js code generator.

ONLY use components from Camped UI (https://ui.camped.academy/docs/components).
NEVER use raw HTML like <input>, <form>, or <button>.

Use Tailwind CSS for layout.
Output valid JSX code for a Next.js page.

For Example, to make a card component with name input field, a framework drop-down menu, deploy and cancel button, the following should be the output:
import {{ Button }} from "@camped-ui/button"
import {{
  Card,
  CardContent,
  CardDescription,
  CardFooter,
  CardHeader,
  CardTitle,
}} from "@camped-ui/card"
import {{ Input }} from "@camped-ui/input"
import {{ Label }} from "@camped-ui/label"
import {{
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
}} from "@camped-ui/select"

export function CardWithForm() {{
  return (
    <Card className="w-[350px]">
      <CardHeader>
        <CardTitle>Create project</CardTitle>
        <CardDescription>Deploy your new project in one-click.</CardDescription>
      </CardHeader>
      <CardContent>
        <form>
          <div className="grid w-full items-center gap-4">
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="name">Name</Label>
              <Input id="name" placeholder="Name of your project" />
            </div>
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="framework">Framework</Label>
              <Select>
                <SelectTrigger id="framework">
                  <SelectValue placeholder="Select" />
                </SelectTrigger>
                <SelectContent position="popper">
                  <SelectItem value="next">Next.js</SelectItem>
                  <SelectItem value="sveltekit">SvelteKit</SelectItem>
                  <SelectItem value="astro">Astro</SelectItem>
                  <SelectItem value="nuxt">Nuxt.js</SelectItem>
                </SelectContent>
              </Select>
            </div>
          </div>
        </form>
      </CardContent>
      <CardFooter className="flex justify-between">
        <Button variant="outline">Cancel</Button>
        <Button>Deploy</Button>
      </CardFooter>
    </Card>
  )
}}

Here are the allowed imports:
{import_block}

### Prompt:
{prompt_text}

### Response:
""".strip()


In [25]:
def run_generation(user_prompt, generator, components_by_package, tag_to_package):
    # Step 1: Infer used components from the user's intent
    used_tags = infer_components_from_prompt(user_prompt, tag_to_package)
    if not used_tags:
        # fallback: use default common imports
        used_tags = ["Button", "Card", "CardContent", "CardHeader", "CardTitle"]

    # Step 2: Create import block
    import_block = generate_imports(used_tags, tag_to_package)

    # Step 3: Build model prompt
    full_prompt = build_prompt(user_prompt, import_block)

    # Step 4: Generate output
    output = generator(full_prompt, max_new_tokens=512, do_sample=True, temperature=0.7, return_full_text=False)
    return output[0]["generated_text"].strip()


In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-instruct",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
    llm_int8_enable_fp32_cpu_offload=True
)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load cheatsheet
components_by_package, tag_to_package, tag_descriptions = parse_cheatsheet("cheatsheet.txt")

# Generate
user_input = "Create a login page with email input, password input, and a submit button"
jsx_code = run_generation(user_input, generator, components_by_package, tag_to_package)
print(jsx_code)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


Here is a simple component for a login page using the components from Camped UI:

```jsx
import { Button } from "@camped-ui/button"
import { Card, CardContent, CardDescription, CardHeader, CardTitle } from "@camped-ui/card"
import { Input } from "@camped-ui/input"
import { Label } from "@camped-ui/label"

export function LoginPage() {
  return (
    <Card className="w-[350px]">
      <CardHeader>
        <CardTitle>Login</CardTitle>
        <CardDescription>Please enter your credentials to access the system.</CardDescription>
      </CardHeader>
      <CardContent>
        <form>
          <div className="grid w-full items-center gap-4">
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="email">Email</Label>
              <Input id="email" placeholder="Your email" />
            </div>
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="password">Password</Label>
              <Input id="password" placeholder="Your pa

In [2]:
# Loading model, tokenizer and setting pipeline for text-generation
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct",
                                          trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct",
                                             trust_remote_code=True,
                                             load_in_8bit=True,
                                             device_map="auto",
                                             # Enable FP32 CPU offload for 8-bit quantization
                                             llm_int8_enable_fp32_cpu_offload=True)


generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


In [16]:
#instructions for the model to follow
instructions = """You are a Next.js code generator.

ONLY use components from Camped UI (https://ui.camped.academy/docs/components).
Import components like this:
  import { Button } from "@camped-ui/button";
  import { Input } from "@camped-ui/input";
  import { Label } from "@camped-ui/label";
  import { Card, CardHeader, CardContent, CardTitle } from "@camped-ui/card";

NEVER use raw HTML like <input>, <form>, or <button>.

✅ Use Tailwind CSS for layout.
✅ Output valid JSX code for Next.js page.

ONLY GIVE THE CODE AS OUTPUT
"""
#user prompt
prompt = """

Generate a home page with heading that says "find yourself here" and button that says "click me" in between
"""
result = f"{instructions}\n {prompt}"
print(result)


You are a Next.js code generator.

ONLY use components from Camped UI (https://ui.camped.academy/docs/components).
Import components like this:
  import { Button } from "@camped-ui/button";
  import { Input } from "@camped-ui/input";
  import { Label } from "@camped-ui/label";
  import { Card, CardHeader, CardContent, CardTitle } from "@camped-ui/card";

NEVER use raw HTML like <input>, <form>, or <button>.

✅ Use Tailwind CSS for layout.
✅ Output valid JSX code for Next.js page.

ONLY GIVE THE CODE AS OUTPUT

 

Generate a home page with heading that says "find yourself here" and button that says "click me" in between



In [17]:
output = generator(result, max_new_tokens=512, do_sample=True, temperature=0.7)
print(output[0]["generated_text"])

You are a Next.js code generator.

ONLY use components from Camped UI (https://ui.camped.academy/docs/components).
Import components like this:
  import { Button } from "@camped-ui/button";
  import { Input } from "@camped-ui/input";
  import { Label } from "@camped-ui/label";
  import { Card, CardHeader, CardContent, CardTitle } from "@camped-ui/card";

NEVER use raw HTML like <input>, <form>, or <button>.

✅ Use Tailwind CSS for layout.
✅ Output valid JSX code for Next.js page.

ONLY GIVE THE CODE AS OUTPUT

 

Generate a home page with heading that says "find yourself here" and button that says "click me" in between
 



 

 

# HomePage

```jsx
import { Button } from "@camped-ui/button";
import { Card, CardHeader, CardContent, CardTitle } from "@camped-ui/card";

export default function HomePage() {
  return (
    <div className="flex flex-col items-center justify-center h-screen">
      <Card>
        <CardHeader>
          <CardTitle>Find Yourself Here</CardTitle>
        </CardH

In [18]:
output

[{'generated_text': 'You are a Next.js code generator.\n\nONLY use components from Camped UI (https://ui.camped.academy/docs/components).\nImport components like this:\n  import { Button } from "@camped-ui/button";\n  import { Input } from "@camped-ui/input";\n  import { Label } from "@camped-ui/label";\n  import { Card, CardHeader, CardContent, CardTitle } from "@camped-ui/card";\n\nNEVER use raw HTML like <input>, <form>, or <button>.\n\n✅ Use Tailwind CSS for layout.\n✅ Output valid JSX code for Next.js page.\n\nONLY GIVE THE CODE AS OUTPUT\n\n \n\nGenerate a home page with heading that says "find yourself here" and button that says "click me" in between\n \n\n\n\n \n\n \n\n# HomePage\n\n```jsx\nimport { Button } from "@camped-ui/button";\nimport { Card, CardHeader, CardContent, CardTitle } from "@camped-ui/card";\n\nexport default function HomePage() {\n  return (\n    <div className="flex flex-col items-center justify-center h-screen">\n      <Card>\n        <CardHeader>\n        